In [ ]:
!pip3 install --user torch
!pip3 install --user torchvision
!pip3 install --user matplotlib
!pip3 install --user decord
!pip3 install --user einops
!pip3 install --user scikit-image
!pip3 install --user pytorch-lightning
from torch.utils.data import DataLoader
!pip3 install gdown


In [ ]:
%cd ViViT

/content/ViViT


In [ ]:
!gdown --id 1--whzasg232ISfR9O_yWAgPwih9utYiL

In [ ]:
!pip install -U torch==1.8.0+cu101 torchvision==0.9.0+cu101 torchtext==0.9.0 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

In [ ]:
%cd ViViT

In [ ]:
import torch
!pip install einops
import einops
from torch import nn, einsum
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from module import Attention, PreNorm, FeedForward
import numpy as np

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        self.norm = nn.LayerNorm(dim)
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return self.norm(x)


  
class ViViT(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, num_frames, dim = 192, depth = 4, heads = 3, pool = 'cls', in_channels = 3, dim_head = 64, dropout = 0.,
                 emb_dropout = 0., scale_dim = 4, ):
        super().__init__()
        
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'
            
        # model = ViViT(224, 16, 400, 16).cuda()


        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = in_channels * patch_size ** 2
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b t c (h p1) (w p2) -> b t (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_frames, num_patches + 1, dim))
        self.space_token = nn.Parameter(torch.randn(1, 1, dim))
        self.space_transformer = Transformer(dim, depth, heads, dim_head, dim*scale_dim, dropout)

        self.temporal_token = nn.Parameter(torch.randn(1, 1, dim))
        self.temporal_transformer = Transformer(dim, depth, heads, dim_head, dim*scale_dim, dropout)

        self.dropout = nn.Dropout(emb_dropout)
        self.pool = pool

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        x = self.to_patch_embedding(x)
        b, t, n, _ = x.shape

        cls_space_tokens = repeat(self.space_token, '() n d -> b t n d', b = b, t=t)
        x = torch.cat((cls_space_tokens, x), dim=2)
        x += self.pos_embedding[:, :, :(n + 1)]
        x = self.dropout(x)

        x = rearrange(x, 'b t n d -> (b t) n d')
        x = self.space_transformer(x)
        x = rearrange(x[:, 0], '(b t) ... -> b t ...', b=b)

        cls_temporal_tokens = repeat(self.temporal_token, '() n d -> b n d', b=b)
        x = torch.cat((cls_temporal_tokens, x), dim=1)

        x = self.temporal_transformer(x)
        

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        return self.mlp_head(x)
    
    
    


    

In [ ]:
!pip install opencv-python-headless
!pip install mmaction2
!pip install timm

In [ ]:
import cv2
import requests
import torch
from torchvision import transforms
from tqdm import tqdm
import os
import sys

import mmaction2
sys.path.insert(1, './ViViT/')
from models.swint_victim import SwinTransformer3D as VICTIM
#from models.swint_student import SwinTransformer3D as STUDENT
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

USE_CUDA = True
PRETRAINED = True
TRAIN = True
epochs = 5
from cv2 import transform
import torchvision
import cv2
#import network
# from approximate_gradients import *
# from cifar10_models import *
import pandas as pd
import os


def measure_true_grad_norm(args, x):
    # Compute true gradient of loss wrt x
    true_grad, _ = compute_gradient(
        args, args.teacher, args.student, x, pre_x=True, device=args.device)

    # PROGRESS UNTIL 13TH MARCH 1:57 AM IST

    true_grad = true_grad.reshape(-1, args.batch_size * 196608)  # 3145728

    # Compute norm of gradients
    norm_grad = true_grad.norm(2, dim=1).mean().cpu()
    return norm_grad

transform_swint = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.Normalize(
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375]
    ),
])



df = pd.read_csv('result400.csv')
ll = sorted(list(df.iloc[:,1].unique()))
dic = {}
for id, i in enumerate(ll):
    dic[i] = id

# dic

# (24 * x) / 16
# 10 * 10s 100 (5 frames)

def video2img(video_path, transform=transform_swint):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    l = []
    fc = 0
    frems = []
    while success:
        frems.append(image)
        success, image = vidcap.read()
    fc = len(frems) 
    
    # print(fc)
    
    if fc == 0:
        print(video_path)
        
    for i in range(0, fc, (fc-1)//15):
        image = frems[i]
        # print(image.shape)
        l.append(
            transform(
                torch.tensor(image).type(
                    torch.FloatTensor).permute(2, 0, 1)
            )
        )
    if len(l) == 0:
        torch.zeros(3, 16, 224, 224)
    else:
        return torch.stack(l[:16], dim=1)


class videosDataset(Dataset):
    
    def __init__(self, csv_file, root_dir, transform=transform_swint):
        
        self.annotations = pd.read_csv(csv_file)
        
        self.root_dir = root_dir
        
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, index):
        vid_path = os.path.join(self.root_dir, (self.annotations.iloc[index, 0]))
        # print(vid_path)
        vid_label = torch.tensor(dic[self.annotations.iloc[index, 1]])
        # put the labels into a dictionary?
        
        vid = video2img(vid_path, self.transform)
        
        # if self.transform:
            # vid = self.transform(vid)
        
        return (vid, vid_label)
        
        



In [ ]:
%cd ViViT

In [ ]:
batch_size = 16
epochs = 5
gen_lr = 1e-4
dis_lr = 1e-4
device = 'cuda'
csv_file = 'result400.csv'
root_dir = './'
rr = './'
data = videosDataset(csv_file, rr, transform = transform_swint)

train_loader = DataLoader(dataset = data, batch_size = batch_size, shuffle = True)
os.listdir(rr)
# for data, targets in (train_loader):

In [ ]:
#   print(data)

def use_pretrained(model,
                   folder='weights/',
                   file_name="swint_victim_pretrained.pth",
                   download=False,
                   url=None, ):
    if download:
        response = requests.get(url, stream=True)
        t = int(response.headers.get('content-length', 0))  # total file size
        block_size = 1024 ** 2  # 1 Mbit
        progress_bar = tqdm(total=t, unit='iB', unit_scale=True)
        with open(f"weights/{file_name}", 'wb') as file:
            for data in response.iter_content(block_size):
                progress_bar.update(len(data))
                file.write(data)
        progress_bar.close()
        if (t != 0) and (progress_bar.n != t):
            print("ERROR downloading weights!")
            return -1
        print(f"Weights downloaded in {folder} directory!")
    model.load_state_dict(torch.load(os.path.join(folder, file_name)))
    return model



#student_model = STUDENT()
victim_model = VICTIM()
if PRETRAINED:
    victim_model = use_pretrained(victim_model)
if USE_CUDA:
    #student_model.cuda()
    victim_model.cuda()
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    #student_model.parameters(),
    model.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    weight_decay=0.02
)

victim_model.eval()

In [ ]:
losses = []
model = ViViT(224, 16, 400, 16).cuda()
for epoch in range(epochs):
    """
    This will run until query is being generated by the generator or else generator \
    will change TRAIN to False.
    """
    # Converting video to image of size [BatchSize=1, channels=3, frames, height=224, width=224]
    # image = video2img(VIDEO_PATH)
    #image = video2img(data_path)

    for image, targets in (train_loader):
        
        # try:
          
          # if __name__ == "__main__":
        img = image.permute(0, 2, 1, 3, 4)
        img = img.cuda()
        #img = torch.ones([1, 13, 3, 224, 224]).cuda()

        # parameters = filter(lambda p: p.requires_grad, model.parameters())
        # parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
        # print('Trainable Parameters: %.3fM' % parameters)
        out = model(img)
          #out = torch.nn.functional.softmax(out)

            # print("Shape of out :", out.shape)      # [B, num_classes]
          #if USE_CUDA:
        image = image.cuda()
        #image = torch.ones([1, 3, 16, 224, 224]).cuda()
        # Querying the label from victim model
        with torch.no_grad():
            LABEL = torch.argmax(victim_model(image), 1).cuda()
        #target = torch.zeros(1, 400)  # (BatchSize, Classes)
        #target[:, LABEL] = 1

        # LABEL = LABEL
        # Training the student model
        #output = student_model(image)
        loss = loss_func(out, LABEL)
        # print(loss)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        losses.append(loss)
        
    print('loss at epoch %d =', epoch, sum(losses)/(len(losses))
         )


In [ ]:
from google.colab import files
files.download("/content/ViViT.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>